In [5]:
# environment setting
import os
import sys
from pathlib import Path

# this should be learning_models
curr_dir = os.getcwd()

# this should be pyhealth, which is two level up from learning_models library
root_dir = Path(curr_dir).parents[1]
os.chdir(root_dir)

sys.path.append(root_dir)

In [7]:
from pyhealth.data.expdata_generator import imagedata as expdata_generator

# override here to specify where the data locates
# root_dir = ''
data_dir = os.path.join(root_dir, 'datasets', 'image')

expdata_id = '2020.0810.image'

# set up the datasets
cur_dataset = expdata_generator(expdata_id, root_dir=root_dir)
cur_dataset.get_exp_data(sel_task='diagnose', data_root=data_dir)
cur_dataset.load_exp_data()
# cur_dataset.show_data()

100%|██████████| 100/100 [00:01<00:00, 93.11it/s]


Current ExpData_ID: 2020.0810.image --- Target for CMS
finished X generate
finished Y generate
generate finished
target Task: diagnose
N of labels: 3
N of TrainData: 64
N of ValidData: 16
N of TestData: 20
load finished
target Task: diagnose
N of labels: 3
N of TrainData: 64
N of ValidData: 16
N of TestData: 20


In [8]:
from pyhealth.models.image.typicalcnn import TypicalCNN as model
# initialize the model for training
expmodel_id = '2020.0810.cnn.image.diagnose.'
clf = model(expmodel_id=expmodel_id, n_epoch=100, use_gpu=True,
            gpu_ids='0,1')
clf.fit(cur_dataset.train, cur_dataset.valid)

# load the best model for inference
clf.load_model()
clf.inference(cur_dataset.test)
results = clf.get_results()
print(results)

use 2 GPUs recource
current task can beed seen as multiclass
=> creating model 'resnet18'
    Total params: 11.18M
=> creating model 'resnet18'
    Total params: 11.18M
load best-th epoch model
current task can beed seen as multiclass
{'hat_y': array([[0.34585968, 0.34950513, 0.30463523],
       [0.3371851 , 0.339385  , 0.32342988],
       [0.33524728, 0.3321462 , 0.3326065 ],
       [0.34149212, 0.32257894, 0.3359289 ],
       [0.35077342, 0.3396097 , 0.30961686],
       [0.35468316, 0.32482758, 0.32048926],
       [0.35699597, 0.33426023, 0.3087438 ],
       [0.35296798, 0.33218038, 0.31485164],
       [0.33812657, 0.34893402, 0.3129394 ],
       [0.33851597, 0.341828  , 0.31965604],
       [0.35516188, 0.3376512 , 0.307187  ],
       [0.34562922, 0.34481993, 0.30955082],
       [0.34262735, 0.34716234, 0.3102103 ],
       [0.34689385, 0.3238576 , 0.32924852],
       [0.3396384 , 0.33442318, 0.3259385 ],
       [0.35435236, 0.32238114, 0.32326648],
       [0.32515126, 0.35782436, 0.3

tr=>epoch=99 Valid Loss: 21.487, Train Loss: 21.485: 100%|██████████| 100/100 [02:46<00:00,  1.66s/it]


In [9]:
from pyhealth.evaluation.evaluator import func
# evaluate the model
r = func(results['hat_y'], results['y'])
print(r)

current data evaluate using multiclass evaluation-type
{'avg_precision_micro': 0.4080676029940976, 'roc_auc_score_micro': 0.5475, 'coverage_error': 1.95, 'label_ranking_average_precision_score': 0.6416666666666668, 'label_ranking_loss': 0.475, 'hamming_loss@1': 0.4, 'recall@1': 0.4, 'precision@1': 0.4, 'hamming_loss@3': 0.6666666666666666, 'recall@3': 1.0, 'precision@3': 0.33333333333333326}
